In [2]:
import pandas as pd
import numpy as np
import tejapi

In [3]:
tejapi.ApiConfig.api_base = "http://10.10.10.66"
tejapi.ApiConfig.api_key = "rCQ1mgb1y8bIgShFl5lLinrXWhjwvD"

# Database 
企業社會責任 > TESG事件雷達 > TWN/AEWATCHA

In [3]:
df = tejapi.get('TWN/AEWATCHA',
                opts={'columns': ['coid', 'mdate', 'issue_main', 'fg_1', 'intensity']},
                paginate=True)

In [4]:
df

,coid,mdate,issue_main,fg_1,intensity
None,,,,,
0,000019,2018-02-05 00:00:00+00:00,S2 資料安全,,-2
1,000020,2022-04-30 00:00:00+00:00,S4 員工資訊統計,,-1
2,000022,2022-01-31 00:00:00+00:00,S4 員工資訊統計,,-1
3,000026,2021-02-08 00:00:00+00:00,S3 產品品質及安全,,-1
4,000055,2019-07-11 00:00:00+00:00,G2 管理領導,,-2
...,...,...,...,...,...
51049,Z4879,2022-10-25 00:00:00+00:00,G3 控制權與席次,,0
51050,Z4898,2019-04-03 00:00:00+00:00,S2 資料安全,,-2
51051,Z4898,2022-07-21 00:00:00+00:00,S2 資料安全,,-2


# Cleaning data

In [5]:
data = df.copy()

In [6]:
data.rename(columns={'coid': 'company_id', 'mdate': 'publication_date', 'intensity': 'severity','fg_1': 'follow-up'}, inplace=True)

In [7]:
data["publication_date"] = pd.to_datetime(data["publication_date"]).dt.date
data['reach'] = 1

In [8]:
ind = pd.read_excel("data/公司屬性及所屬產業_20221012.xlsx")
# 取金控代號前四位數
ind['code'] = ind['code'].apply(lambda x: str(x)[:4])

In [9]:
# 將證券新聞併入到金控底下
data_new = pd.merge(data, ind, on='company_id', how="left")

In [10]:
# 如果 code 為 Nan 保持原本 id ，如 code 有數字則替換成 code
data_new['company_id_new'] = np.where(data_new['code'].isnull(), data_new['company_id'], data_new['code'])

In [11]:
df_Unique = data_new[data_new['follow-up']!='Y'].copy()
df_Unique.loc[:,'novelty'] = 2
df_Y = data_new[data_new['follow-up']=='Y'].copy()
df_Y.loc[:,'novelty'] = 1

data_total = pd.concat([df_Unique, df_Y])

In [12]:
data_total.drop(['company_id', 'company_name', 'code', 'follow-up'], axis=1, inplace=True)
data_total.rename(columns={'company_id_new': 'company_id'}, inplace=True)

In [13]:
ind = pd.read_csv("data/SASBindustry.csv")
ind.company_id = ind.company_id.apply(lambda x: str(x))

In [14]:
indDictName = ind.set_index(['company_id'])['company_name'].to_dict()
indDictSASB = ind.set_index(['company_id'])['SASB'].to_dict()

In [15]:
def tryconvert(value):
    try:
        indDictName[str(value)]
        return indDictName[str(value)]
    except (KeyError):
        return None

def tryconvert2(value):
    try:
        indDictSASB[str(value)]
        return indDictSASB[str(value)]
    except (KeyError):
        return None

data_total['company_name'] = data_total['company_id'].apply(lambda x: tryconvert(str(x)))
data_total['SASB'] = data_total['company_id'].apply(lambda x: tryconvert2(str(x)))
data_total['company_id'] = data_total['company_id'].apply(lambda x: str(x))

In [16]:
data_total = data_total[data_total['company_name'].notna()]
data_total = data_total[data_total['SASB'].notna()]

In [17]:
data_total = data_total.sort_values(by=['company_id', 'publication_date'])

In [18]:
data_total = data_total.reset_index(drop=True)

In [20]:
data_total

,publication_date,issue_main,severity,reach,company_id,novelty,company_name,SASB
0,2018-01-31,E4 廢棄物及有毒物質管理,-3,1,1101,2,台泥,提煉與礦產加工
1,2018-02-28,E4 廢棄物及有毒物質管理,-1,1,1101,2,台泥,提煉與礦產加工
2,2018-03-31,E4 廢棄物及有毒物質管理,-1,1,1101,2,台泥,提煉與礦產加工
3,2018-05-31,E4 廢棄物及有毒物質管理,-1,1,1101,2,台泥,提煉與礦產加工
4,2018-06-22,G3 控制權與席次,0,1,1101,2,台泥,提煉與礦產加工
...,...,...,...,...,...,...,...,...
47370,2021-12-31,E4 廢棄物及有毒物質管理,-1,1,9986,2,南天有線,服務
47371,2022-06-09,G3 控制權與席次,0,1,9986,2,南天有線,服務
47372,2022-06-13,G3 控制權與席次,0,1,9986,2,南天有線,服務
47373,2022-07-28,G3 控制權與席次,0,1,9986,2,南天有線,服務


In [21]:
data_total = data_total[['company_id', 'company_name', 'SASB', 'publication_date', 'issue_main', 'severity', 'reach', 'novelty' ]]

In [22]:
data_total

,company_id,company_name,SASB,publication_date,issue_main,severity,reach,novelty
0,1101,台泥,提煉與礦產加工,2018-01-31,E4 廢棄物及有毒物質管理,-3,1,2
1,1101,台泥,提煉與礦產加工,2018-02-28,E4 廢棄物及有毒物質管理,-1,1,2
2,1101,台泥,提煉與礦產加工,2018-03-31,E4 廢棄物及有毒物質管理,-1,1,2
3,1101,台泥,提煉與礦產加工,2018-05-31,E4 廢棄物及有毒物質管理,-1,1,2
4,1101,台泥,提煉與礦產加工,2018-06-22,G3 控制權與席次,0,1,2
...,...,...,...,...,...,...,...,...
47370,9986,南天有線,服務,2021-12-31,E4 廢棄物及有毒物質管理,-1,1,2
47371,9986,南天有線,服務,2022-06-09,G3 控制權與席次,0,1,2
47372,9986,南天有線,服務,2022-06-13,G3 控制權與席次,0,1,2
47373,9986,南天有線,服務,2022-07-28,G3 控制權與席次,0,1,2


# Output

In [23]:
data_total.to_csv("output/TESG.csv", index=0, encoding='utf_8_sig')

# w/ severity = 0 

In [4]:
df = pd.read_csv("data/TESG.csv")
df

,company_id,company_name,SASB,publication_date,issue_main,severity,reach,novelty
0,1101,台泥,提煉與礦產加工,2018/1/31,E4 廢棄物及有毒物質管理,-3,1,2
1,1101,台泥,提煉與礦產加工,2018/2/28,E4 廢棄物及有毒物質管理,-1,1,2
2,1101,台泥,提煉與礦產加工,2018/3/31,E4 廢棄物及有毒物質管理,-1,1,2
3,1101,台泥,提煉與礦產加工,2018/5/31,E4 廢棄物及有毒物質管理,-1,1,2
4,1101,台泥,提煉與礦產加工,2018/6/22,G3 控制權與席次,0,1,2
...,...,...,...,...,...,...,...,...
47370,9986,南天有線,服務,2021/12/31,E4 廢棄物及有毒物質管理,-1,1,2
47371,9986,南天有線,服務,2022/6/9,G3 控制權與席次,0,1,2
47372,9986,南天有線,服務,2022/6/13,G3 控制權與席次,0,1,2
47373,9986,南天有線,服務,2022/7/28,G3 控制權與席次,0,1,2


In [5]:
df[df['severity']!=0].to_csv("data/TESGwithout0.csv", index=0, encoding='utf_8_sig')